# Dataset Description

## awards_players

- playerId
- award (should standardize and abreviate)
- Lg ID (remove)
- year

## coaches

- lgId (remove)
- Stint (timeperiod coach was at the team) (use to convert years)
- Post_wins e Post_loses (playoff wins, irrelevant, remove)
  
## players_teams

- Stint (timeperiod player was in the team)
- lgId (remove)
- GP (games played)
- GS (game started)
- ORebounds (offensive rebounds)
- DRebounds (defensive rebounds)
- rebounds (total) (check sum of previous 2)
- turnovers (negative stat)
- PF (Personal Fouls)
- Fg Attempted (field goals attempted)
- Fg Made (maybe change to ratio)
- Ft (free throws)
- Dq (desqualify)
- Post ... (maybe ignore)

## Players

- POS (postion, has missing value) (Center, Forward-Center, Guard, Forward-Guard) (check if multiple positions over time)
- firstseason e lastseason (remove)
- height (missing values) (check if same units)
- college (some missing values, some none)
- death date (ignore dead players)

## series and teams post
- ignore files, reffering to the season after playoffs qualification  

## teams

- TmId (team Id), Franch ID (Franchise Id) (maybe irrelevant, maybe not)
- DivId (remove, all empty)
- Rank (maybe seed)
- Seeded (remove)
- **Playoff** (N, Y) to predict next
- first round, semis, finals (change to one column that has numbers that relate to when they got eliminated) (label0-no playoffs, label1-loss on first round, label2-loss on semis, label3-loss on finals, label4-no losses)
- Name (already have ID, may not be necessary)
- O_fgm (offensive field goals made)
- O_fga (offensive field goals attempted)
- Tm ORB, Tm DRB, Tm TRB ,Oppmt ORS, Oppmt DRB, Oppmt TRB (all 0, remove)
- GP (all teams play all games, maybe remove column)
- Win Loss -> create ratio
- Conf W L (maybe irrelevent)
- Attend (number of spectators, maybe irrelevant)
- Arena (remove)


# Developments

## players_teams

(All this was changed via python)
- Got per games for main statistical categories (Points, Assists, Rebounds (Off and Def), Steals, Blocks, Turnovers, Personal Fouls)
- Got percentages for main shots (Field Goals, Free Throws, Three Pointers)
- Removed League ID
- Created File with and without Post Season Performance
- Set all missing values to 0

![Alt text](prntscrns/ptms.png)


## awards_players

(All this was changed via python)
- Removed League ID
- Fixed an incorrectly inputted value (Kim Perrot Sportmanship -> Kim Perrot Sportmanship Award)
- Abbreviated the award names to a standard (Capitalized and First Letters Only)

![Alt text](prntscrns/awrds.png)
